## Sarcasm Headlines Classifier Using Bidirectional LSTM



In [1]:
import pandas as pd

In [2]:
df=pd.read_json('/content/Sarcasm_Headlines_Dataset_v2.json', lines = True)

In [3]:
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [4]:
###Drop Nan Values
df=df.dropna()


In [5]:
## Get the Independent Features

X=df.drop(['article_link','is_sarcastic'],axis=1)

In [6]:
## Get the Dependent features
y=df['is_sarcastic']

In [7]:
y.value_counts()

0    14985
1    13634
Name: is_sarcastic, dtype: int64

In [8]:
X.shape

(28619, 1)

In [9]:
y.shape

(28619,)

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.8.2'

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [13]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [14]:
messages=X.copy()

In [15]:
messages['headline'][1]

'dem rep. totally nails why congress is falling short on gender, racial equality'

In [16]:
messages.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['headline'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
23619
23620
23621
23622
23623
23624
23625
23626
23627
23628
23629
23630
23631
23632
23633
23634
23635
23636
23637
23638
23639
23640
23641
23642
23643
23644
23645
23646
23647
23648
23649
23650
23651
23652
23653
23654
23655
23656
23657
23658
23659
23660
23661
23662
23663
23664
23665
23666
23667
23668
23669
23670
23671
23672
23673
23674
23675
23676
23677
23678
23679
23680
23681
23682
23683
23684
23685
23686
23687
23688
23689
23690
23691
23692
23693
23694
23695
23696
23697
23698
23699
23700
23701
23702
23703
23704
23705
23706
23707
23708
23709
23710
23711
23712
23713
23714
23715
23716
23717
23718
23719
23720
23721
23722
23723
23724
23725
23726
23727
23728
23729
23730
23731
23732
23733
23734
23735
23736
23737
23738
23739
23740
23741
23742
23743
23744
23745
23746
23747
23748
23749
23750
23751
23752
23753
23754
23755
23756
23757
23758
23759
23760
23761
23762
23763
23764
23765
23766
23767
23768
23769
23770
23771
23772
23773
23774
23775
23776
2

In [20]:
corpus

['thirtysometh scientist unveil doomsday clock hair loss',
 'dem rep total nail congress fall short gender racial equal',
 'eat veggi delici differ recip',
 'inclement weather prevent liar get work',
 'mother come pretti close use word stream correctli',
 'white inherit',
 'way file tax less stress',
 'richard branson global warm donat nearli much cost fail balloon trip',
 'shadow govern get larg meet marriott confer room b',
 'lot parent know scenario',
 'lesbian consid father indiana amaz one',
 'amanda peet told daughter sex special hug',
 'know regard current treatment ebola',
 'chri christi suggest hillari clinton blame boko haram kidnap hundr schoolgirl',
 'ford develop new suv run pure gasolin',
 'uber ceo travi kalanick step trump econom advisori council',
 'area boy enter jump touch top doorway phase',
 'area man travel gurney',
 'leav person disabl behind',
 'lin manuel miranda would like remind put phone away',
 'journalist kill target intern press rise',
 'guard video game 

In [21]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1225, 1506, 2515, 2134, 3092, 4563, 2236],
 [3223, 4220, 3466, 4615, 690, 161, 3597, 1903, 1515, 1622],
 [4596, 2915, 2487, 496, 4332],
 [3743, 1956, 579, 2941, 467, 2411],
 [4257, 4516, 2907, 2053, 4249, 1695, 454, 2148],
 [1713, 3568],
 [2164, 4693, 2079, 250, 2961],
 [4742, 2626, 967, 4473, 1143, 2152, 331, 3811, 777, 4285, 1957],
 [3789, 288, 467, 4439, 3492, 4091, 44, 1039, 3594],
 [328, 1227, 4709, 4072],
 [849, 3807, 1748, 2468, 915, 299],
 [1217, 2241, 4917, 4878, 178, 1161, 42],
 [4709, 208, 4663, 4148, 4567],
 [4828, 1226, 4493, 2972, 2884, 699, 1940, 3631, 4876, 551, 381],
 [2651, 2617, 2606, 2916, 4739, 858, 1502],
 [3487, 1844, 4132, 2846, 1168, 3292, 845, 4550, 485],
 [814, 3417, 2411, 773, 1041, 2120, 3300, 4443],
 [814, 3967, 658, 722],
 [4135, 2205, 663, 2054],
 [3876, 4491, 4635, 4546, 332, 2047, 1069, 3673, 3419],
 [2042, 791, 657, 3040, 2168, 871],
 [4585, 2054, 1315, 4692, 2338, 2889, 2394, 225, 4043],
 [4356],
 [2426, 1399, 4590, 4729, 3818],
 [735, 2884, 2615, 

### Embedding Representation

In [22]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3092 4563 2236]
 [   0    0    0 ... 1903 1515 1622]
 [   0    0    0 ... 2487  496 4332]
 ...
 [   0    0    0 ... 2815 2379 1804]
 [   0    0    0 ...  130 4106  366]
 [   0    0    0 ... 2875 1986 4081]]


In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 1225, 1506, 2515, 2134, 3092, 4563, 2236], dtype=int32)

In [24]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
len(embedded_docs),y.shape

(28619, (28619,))

In [27]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [28]:
X_final.shape,y_final.shape

((28619, 20), (28619,))

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [30]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
300/300 [==============================] - 20s 54ms/step - loss: 0.5560 - accuracy: 0.7017 - val_loss: 0.5020 - val_accuracy: 0.7469
Epoch 2/10
300/300 [==============================] - 15s 50ms/step - loss: 0.4274 - accuracy: 0.7996 - val_loss: 0.4931 - val_accuracy: 0.7592
Epoch 3/10
300/300 [==============================] - 15s 51ms/step - loss: 0.3730 - accuracy: 0.8331 - val_loss: 0.5177 - val_accuracy: 0.7527
Epoch 4/10
300/300 [==============================] - 15s 49ms/step - loss: 0.3250 - accuracy: 0.8611 - val_loss: 0.5590 - val_accuracy: 0.7511
Epoch 5/10
300/300 [==============================] - 15s 50ms/step - loss: 0.2908 - accuracy: 0.8778 - val_loss: 0.5845 - val_accuracy: 0.7499
Epoch 6/10
300/300 [==============================] - 15s 49ms/step - loss: 0.2583 - accuracy: 0.8940 - val_loss: 0.6536 - val_accuracy: 0.7461
Epoch 7/10
300/300 [==============================] - 15s 49ms/step - loss: 0.2332 - accuracy: 0.9070 - val_loss: 0.6647 - val_accuracy:

### Performance Metrics And Accuracy

In [31]:
y_pred1 = (model.predict(X_test) > 0.5).astype("int32")

In [32]:
from sklearn.metrics import confusion_matrix

In [33]:
confusion_matrix(y_test,y_pred1)

array([[2354, 2562],
       [2302, 2227]])

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.48501852832186343

In [35]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.51      0.48      0.49      4916
           1       0.47      0.49      0.48      4529

    accuracy                           0.49      9445
   macro avg       0.49      0.49      0.48      9445
weighted avg       0.49      0.49      0.49      9445

